In [ ]:
import os
import os.path

import pickle
import pandas as pd
import numpy as np

from tqdm import tqdm

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action="ignore")

# 1. Загрузим ИНН и названия вузов

In [ ]:
with open('..\\data\\raw\\vuz_inn.pickle', 'rb') as f:
        vuz_inn = pickle.load(f)

In [ ]:
vuz_inn.head()

# 2. Посчитаем для каких вузов доступна финансовая информация

Источник данных - сведения о планах финансово-хозяйственной деятельности https://bus.gov.ru/open-data.
Определим, для каких вузов финансовая информация доступна на bus.gov.ru. 
Берем финансовую информацию за все года что есть.

In [ ]:
# %%pycodestyle
data_dir = os.path.dirname(os.getcwd()) + '\\data\\raw'
file_list = os.listdir(data_dir)
if 'years_num.pickle' in file_list:
    print('нашлось сохраненное years_num')
    with open('..\\data\\raw\\years_num.pickle', 'rb') as f:
        years_num = pickle.load(f)
else:
    years_num = []
    for i in tqdm(range(800)):
        inn = vuz_inn['inn'][i]
        url = 'https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn
        response = requests.get(url)
        data = response.json()

        if data['content'] == []:
            years_num.append(0)
        else:
            temp = []
            for i in range(len(data['content'])):
                temp.append(data['content'][i]['common']['financialYear'])
            years_num.append(len(temp))
    with open('..\\data\\raw\\years_num.pickle', 'wb') as f:
        pickle.dump(years_num, f)

vuz_inn['years_num'] = years_num

years_num_unique = vuz_inn['years_num'].unique()

print('Финансовая информация доступна для',
      len(vuz_inn[vuz_inn['years_num'] > 0])*100/800,
      '% ВУЗов')

TOTAL_NUMBER = 800
FIN_INFO_AVAILABLE = len(vuz_inn[vuz_inn['years_num'] > 0])
FIN_INFO_UNAVAILABLE = len(vuz_inn[vuz_inn['years_num'] == 0])

vus_general = {'Финансовая информация доступна': FIN_INFO_AVAILABLE,
               'Финансовая информация НЕдоступна':FIN_INFO_UNAVAILABLE}
vus_general_ser = pd.Series(vus_general) 

# Make the plot with pandas
vus_general_ser.plot(kind='pie', subplots=True)
plt.title("Набор данных по вузам")
plt.ylabel("")
plt.show()

In [ ]:
vuz_inn[['full_name', 'years_num']].head()

# 3. Отбросим ВУЗы, у которых недоступна финансовая информация

In [ ]:
# %%pycodestyle
vuz_inn_fin = vuz_inn[vuz_inn['years_num'] > 0]
vuz_inn_fin.reset_index(inplace=True)
vuz_inn_fin.drop(['index'], axis=1, inplace=True)
print('Осталось', len(vuz_inn_fin), 'вузов')

In [ ]:
no_data = vuz_inn[vuz_inn['years_num'] == 0]['full_name']
print("Для этих ", len(no_data), " вузов нет данных:\n")
n = 1
for i in vuz_inn[vuz_inn['years_num'] == 0]['full_name']:
    print(n, '-', i)
    n += 1

# 4. Загрузим по api финансовые показатели ВУЗов

## 4.1. Блок indicatorsOfFinancialCondition

Будем использовать только обязательные параметры. Потому что необязательные могут быть незаполненными.

**Блок indicatorsOfFinancialCondition. Показатели финансового состояния:**
- sumRealEstate - Недвижимое имущество O
- sumRealEstateResidual - Остаточная стоимость недвижимого имущества O
- sumValuableProperty	- Особо ценное движимое имущество (обязательное)
- sumValuablePropertyResidual - Остаточная стоимость особо ценного движимого имущества O
- sumBalanceNoFinancial - Сумма нефинансовых активов O
- cash - Денежные средства учреждения O
- accountsCash - Денежные средства учреждения на счетах O
- depositCash	- Денежные средства учреждения, размещенные на депозиты в кредитной организации O
- others	- Иные финансовые инструменты O
- debentures - Долговые обязательства O
- kredit - Кредиторская задолженность O
- sumDelayedPayable - Просроченная кредиторская задолженность O

In [ ]:
# %%pycodestyle
def print_block_features_example(block_name):
    """Return a foobang
    
    Optional plotz says to frobnicate the bizbaz first.
    
    """
    inn = '3528051834'
    url = 'https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn
    response = requests.get(url)
    data = response.json()

    indicators_dict = data['content'][0][block_name]
    print('В блоке', block_name,
          'содержится', len(indicators_dict),
          'признака:')
    if block_name == 'indicatorsOfFinancialCondition':
        num = 0
        for i in indicators_dict:
            print(num, '-', i)
            num += 1
        print('\nПример для ИНН =', inn)
        print(indicators_dict)
    else:
        names = []
        num = 0
        for i in indicators_dict:
            print(num, '-', i['name'])
            names.append(i['name'])
            num += 1
        print('\nПример для ИНН =', inn)
        for i in indicators_dict:
            print(i, '\n')
        return names

In [ ]:
indicators_of_financial_condition_names = print_block_features_example('indicatorsOfFinancialCondition')

In [ ]:
def dict2df(indicators_dict, inn, year, last_update):
    prepared_dict = {i: x for i, x in enumerate(indicators_dict.items())}
    df = pd.DataFrame.from_dict(prepared_dict)
    df.columns = df.T[0].tolist()
    df.drop(0, inplace=True)
    df['inn'] = inn
    df['year'] = year
    df['last_update'] = last_update
    df['uniq'] = str(inn)+str(year)+str(last_update)
    return df

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\raw'
file_list = os.listdir(data_dir)
if 'indicators_of_financial_condition_year_inn.pickle' in file_list:
    print('нашлось сохраненное indicators_of_financial_condition_year_inn')
    with open('..\\data\\raw\\indicators_of_financial_condition_year_inn.pickle', 'rb') as f:
        fin_indicators = pickle.load(f)
    fin_indicators.reset_index(inplace=True)
    fin_indicators.drop(['index'], axis=1, inplace=True)
else:
    fin_indicators = pd.DataFrame(columns=['sumRealEstate', 'sumRealEstateResidual', 'sumValuableProperty',
       'sumValuablePropertyResidual', 'sumBalanceNoFinancial', 'cash',
       'accountsCash', 'depositCash', 'others', 'sumDepthIncome',
       'sumDepthExpenses', 'sumFinancialActives', 'debentures', 'kredit',
       'sumDelayedPayable', 'sumObligations', 'inn', 'year', 'last_update'])
    
    for i in tqdm(range(len(vuz_inn_fin))):
        inn = vuz_inn_fin['inn'][i]
        #print(inn)
        response = requests.get('https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn)
        data = response.json()
        
        #last_update_date = last_update_date_checking(data)
        #print(last_update_date)
        
        #year_memory = [] # Например для 7303002000 несколько обновлений данный за один год. 
        # Не понятно как брать актуальное, так как дата обновления одна и та же. Берем первое вхождение, остальные пропускаем.
        for j in range(len(data['content'])):
            year = data['content'][j]['common']['financialYear']
            last_update = data['content'][j]['common']['lastUpdate']
            #last_update_for_year = last_update_date[year]
            indicators_of_financial_condition = data['content'][j]['indicatorsOfFinancialCondition']
            temp_df = dict2df(indicators_of_financial_condition, inn, year, last_update)
            fin_indicators = fin_indicators.append(temp_df.iloc[0])


    with open('..\\data\\raw\\indicators_of_financial_condition_year_inn.pickle', 'wb') as f:
            pickle.dump(fin_indicators, f)

In [ ]:
fin_indicators.head(5)

In [ ]:
print('Итак, мы имеем для ',  len(vuz_inn_fin), ' вузов загруженные финансовые показатели за разные года.\n', \
      'Все они собраны в ', len(fin_indicators), 'записи (строки).')

In [ ]:
fin_indicators.info()

Видно, что во всех столбцах, кроме inn, year, last_update есть пропущенные и нулевые значения.

In [ ]:
# удалим необязательные показатели
optional_columns = ['sumDepthIncome', 'sumDepthExpenses', 'sumFinancialActives', 'sumObligations']
fin_indicators.drop(optional_columns, axis=1, inplace=True)

fin_indicators.reset_index(inplace=True)
fin_indicators.drop(['index'], axis=1, inplace=True)

fin_indicators.head()

In [ ]:
# посмотрим, сколько записей есть для каждого года
fin_indicators.groupby(['year']).count()['sumRealEstate'].to_dict()

## 4.2. Блок planPaymentIndexes
Показатели по поступлениям и выплатам учреждения (подразделения) (с 2017):



In [ ]:
plan_payment_indexes_names = print_block_features_example('planPaymentIndexes')

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\raw'
file_list = os.listdir(data_dir)
if 'plan_payment_indexes_df.pickle' in file_list:
    print('нашлось сохраненное plan_payment_indexes_df')
    with open('..\\data\\raw\\plan_payment_indexes_df.pickle', 'rb') as f:
        plan_payment_indexes_df = pickle.load(f)
    plan_payment_indexes_df.reset_index(inplace=True)
    plan_payment_indexes_df.drop(['index'], axis=1, inplace=True)
else:
    plan_payment_indexes_df = pd.DataFrame(columns=['plan_payment_indexes_0', 'plan_payment_indexes_1', 
                                           'plan_payment_indexes_2', 'plan_payment_indexes_3', 
                                           'plan_payment_indexes_4', 'plan_payment_indexes_5', 
                                           'plan_payment_indexes_6', 'plan_payment_indexes_7', 
                                           'plan_payment_indexes_8', 'plan_payment_indexes_9', 
                                           'plan_payment_indexes_10', 'plan_payment_indexes_11', 
                                           'plan_payment_indexes_12', 'plan_payment_indexes_13', 
                                           'plan_payment_indexes_14', 'plan_payment_indexes_15', 
                                           'plan_payment_indexes_16', 'plan_payment_indexes_17', 
                                           'plan_payment_indexes_18', 'plan_payment_indexes_19', 
                                           'plan_payment_indexes_20', 'plan_payment_indexes_21', 
                                           'plan_payment_indexes_22', 'plan_payment_indexes_23'])

    for i in tqdm(range(len(vuz_inn_fin))):
        inn = vuz_inn_fin['inn'][i]
        response = requests.get('https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn)
        data = response.json()

        for j in range(len(data['content'])):
            year = data['content'][j]['common']['financialYear']
            last_update = data['content'][j]['common']['lastUpdate']

            plan_payment_indexes = data['content'][j]['planPaymentIndexes']

            index_dict = dict.fromkeys(['plan_payment_indexes_' + str(kk) for kk in range(24)], np.nan)
            for indexes in plan_payment_indexes:
                num = 0
                for index_names in plan_payment_indexes_names:
                    if indexes['name']==index_names:
                        index_dict['plan_payment_indexes_' + str(num)] = indexes['total']
                        num+=1

            temp_df = dict2df(index_dict, inn, year, last_update)
            plan_payment_indexes_df = plan_payment_indexes_df.append(temp_df.iloc[0])


    with open('..\\data\\raw\\plan_payment_indexes_df.pickle', 'wb') as f:
                pickle.dump(plan_payment_indexes_df, f)

# 4.3. Блок planPaymentIndexesMain

In [ ]:
plan_payment_indexes_main_names = print_block_features_example('planPaymentIndexesMain')

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\raw'
file_list = os.listdir(data_dir)
if 'plan_payment_indexes_main_df.pickle' in file_list:
    print('нашлось сохраненное plan_payment_indexes_main_df')
    with open('..\\data\\raw\\plan_payment_indexes_main_df.pickle', 'rb') as f:
        plan_payment_indexes_main_df = pickle.load(f)
    plan_payment_indexes_main_df.reset_index(inplace=True)
    plan_payment_indexes_main_df.drop(['index'], axis=1, inplace=True)
else:
    plan_payment_indexes_main_df = pd.DataFrame(columns=['plan_payment_indexes_main_0', 'plan_payment_indexes_main_1', 
                                           'plan_payment_indexes_main_2', 'plan_payment_indexes_main_3', 
                                           'plan_payment_indexes_main_4', 'plan_payment_indexes_main_5'])

    for i in tqdm(range(len(vuz_inn_fin))):
        inn = vuz_inn_fin['inn'][i]
        response = requests.get('https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn)
        data = response.json()

        for j in range(len(data['content'])):
            year = data['content'][j]['common']['financialYear']
            last_update = data['content'][j]['common']['lastUpdate']

            plan_payment_indexes_main = data['content'][j]['planPaymentIndexesMain']

            index_dict = dict.fromkeys(['plan_payment_indexes_main_' + str(kk) for kk in range(6)], np.nan)
            for indexes in plan_payment_indexes_main:
                num = 0
                for index_names in plan_payment_indexes_main_names:
                    if indexes['name']==index_names:
                        index_dict['plan_payment_indexes_main_' + str(num)] = indexes['total']
                        num+=1

            temp_df = dict2df(index_dict, inn, year, last_update)
            plan_payment_indexes_main_df = plan_payment_indexes_main_df.append(temp_df.iloc[0])


    with open('..\\data\\raw\\plan_payment_indexes_main_df.pickle', 'wb') as f:
                pickle.dump(plan_payment_indexes_main_df, f)

## Блок expensePaymentIndexes

In [ ]:
expense_payment_indexes_names = print_block_features_example('expensePaymentIndexes')

In [ ]:
index_name_list = []
for indexes_num in range(3):
    index_name_list.append('nextYearFz44Sum' + str(indexes_num))
    index_name_list.append('nextYearFz223Sum' + str(indexes_num))
    index_name_list.append('nextYearTotalSum' + str(indexes_num))
    index_name_list.append('firstPlanYearFz44Sum' + str(indexes_num))
    index_name_list.append('firstPlanYearFz223Sum' + str(indexes_num))
    index_name_list.append('firstPlanYearTotalSum' + str(indexes_num))
    index_name_list.append('secondPlanYearFz44Sum' + str(indexes_num))
    index_name_list.append('secondPlanYearFz223Sum' + str(indexes_num))
    index_name_list.append('secondPlanYearTotalSum' + str(indexes_num))
index_name_list

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\raw'
file_list = os.listdir(data_dir)
if 'expense_pyment_indexes_df.pickle' in file_list:
    print('нашлось сохраненное expense_pyment_indexes_df')
    with open('..\\data\\raw\\expense_pyment_indexes_df.pickle', 'rb') as f:
        expense_pyment_indexes_df = pickle.load(f)
    expense_pyment_indexes_df.reset_index(inplace=True)
    expense_pyment_indexes_df.drop(['index'], axis=1, inplace=True)
else:
    expense_pyment_indexes_df = pd.DataFrame(columns=index_name_list)
    
    for i in tqdm(range(len(vuz_inn_fin))):
        inn = vuz_inn_fin['inn'][i]
        
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        url = 'https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn
        response = session.get(url)
        #response = requests.get()
        data = response.json()

        for j in range(len(data['content'])):
            year = data['content'][j]['common']['financialYear']
            last_update = data['content'][j]['common']['lastUpdate']

            expense_pyment_indexes = data['content'][j]['expensePaymentIndexes']

            index_dict = dict.fromkeys(index_name_list, np.nan)
            
            for indexes in expense_pyment_indexes:  
                for indexes_num in range(3):
                    if indexes['name']==expense_payment_indexes_names[indexes_num]:
                        index_dict['nextYearFz44Sum' + str(indexes_num)] = indexes['nextYearFz44Sum']
                        index_dict['nextYearFz223Sum' + str(indexes_num)] = indexes['nextYearFz223Sum']
                        index_dict['nextYearTotalSum' + str(indexes_num)] = indexes['nextYearTotalSum']
                        index_dict['firstPlanYearFz44Sum' + str(indexes_num)] = indexes['firstPlanYearFz44Sum']
                        index_dict['firstPlanYearFz223Sum' + str(indexes_num)] = indexes['firstPlanYearFz223Sum']
                        index_dict['firstPlanYearTotalSum' + str(indexes_num)] = indexes['firstPlanYearTotalSum']
                        index_dict['secondPlanYearFz44Sum' + str(indexes_num)] = indexes['secondPlanYearFz44Sum']
                        index_dict['secondPlanYearFz223Sum' + str(indexes_num)] = indexes['secondPlanYearFz223Sum']
                        index_dict['secondPlanYearTotalSum' + str(indexes_num)] = indexes['secondPlanYearTotalSum']

            temp_df = dict2df(index_dict, inn, year, last_update)
            expense_pyment_indexes_df = expense_pyment_indexes_df.append(temp_df.iloc[0])


    with open('..\\data\\raw\\expense_pyment_indexes_df.pickle', 'wb') as f:
                pickle.dump(expense_pyment_indexes_df, f)

In [ ]:
expense_pyment_indexes_df.head()

## Блок temporaryResourcesList

In [ ]:
temporary_resources_list_names = temporary_resources_list_indexes_names = print_block_features_example('temporaryResourcesList')

In [ ]:
data_dir = os.path.dirname(os.getcwd()) + '\\data\\raw'
file_list = os.listdir(data_dir)
if 'temporary_resources_list_df.pickle' in file_list:
    print('нашлось сохраненное temporary_resources_list_df')
    with open('..\\data\\raw\\temporary_resources_list_df.pickle', 'rb') as f:
        temporary_resources_list_df = pickle.load(f)
    temporary_resources_list_df.reset_index(inplace=True)
    temporary_resources_list_df.drop(['index'], axis=1, inplace=True)
else:
    temporary_resources_list_df = pd.DataFrame(columns=['temporary_resources_list_0', 'temporary_resources_list_1', 
                                           'temporary_resources_list_2', 'temporary_resources_list_3'])

    for i in tqdm(range(len(vuz_inn_fin))):
        inn = vuz_inn_fin['inn'][i]
        
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        url = 'https://bus.gov.ru/public-rest/api/epbs/fap.json?inn=' + inn
        response = session.get(url)
        data = response.json()

        for j in range(len(data['content'])):
            year = data['content'][j]['common']['financialYear']
            last_update = data['content'][j]['common']['lastUpdate']

            temporary_resources_list = data['content'][j]['temporaryResourcesList']

            index_dict = dict.fromkeys(['temporary_resources_list_' + str(kk) for kk in range(len(temporary_resources_list_names))], np.nan)
            for indexes in temporary_resources_list:
                num = 0
                for index_names in temporary_resources_list_names:
                    if indexes['name']==index_names:
                        index_dict['temporary_resources_list_' + str(num)] = indexes['total']
                        num+=1

            temp_df = dict2df(index_dict, inn, year, last_update)
            temporary_resources_list_df = temporary_resources_list_df.append(temp_df.iloc[0])


    with open('..\\data\\raw\\temporary_resources_list_df.pickle', 'wb') as f:
                pickle.dump(temporary_resources_list_df, f)

# Объединим все заруженные данные в один df
- fin_indicators
- plan_payment_indexes_df
- plan_payment_indexes_main_df
- expense_pyment_indexes_df
- temporary_resources_list_df

In [ ]:
print(fin_indicators.shape, \
     plan_payment_indexes_df.shape, \
     plan_payment_indexes_main_df.shape, \
     expense_pyment_indexes_df.shape, \
     temporary_resources_list_df.shape)

In [ ]:
sum(plan_payment_indexes_df['uniq']==fin_indicators['uniq'])

In [ ]:
plan_payment_indexes_df.drop(['inn', 'year', 'last_update', 'uniq'], inplace=True, axis=1)
plan_payment_indexes_main_df.drop(['inn', 'year', 'last_update', 'uniq'], inplace=True, axis=1)
expense_pyment_indexes_df.drop(['inn', 'year', 'last_update', 'uniq'], inplace=True, axis=1)
temporary_resources_list_df.drop(['inn', 'year', 'last_update', 'uniq'], inplace=True, axis=1)

In [ ]:
all_indicators = pd.concat([fin_indicators, \
                            plan_payment_indexes_df, \
                            plan_payment_indexes_main_df, \
                            expense_pyment_indexes_df, \
                            temporary_resources_list_df], axis=1)

In [ ]:
all_indicators.drop(['uniq'], inplace=True, axis=1)

In [ ]:
all_indicators.shape

In [ ]:
all_indicators.columns

In [ ]:
with open('..\\data\\raw\\all_indicators.pickle', 'wb') as f:
                pickle.dump(all_indicators, f)